<a href="https://colab.research.google.com/github/emmenets/neuralnetwork/blob/master/210525_objectdetectionclassifier_tflite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
cd /gdrive/MyDrive/colabfiles/tf2programmfolder

/gdrive/MyDrive/colabfiles/tf2programmfolder


In [4]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 57367, done.
remote: Counting objects: 100% (1430/1430), done.
remote: Compressing objects: 100% (615/615), done.
remote: Total 57367 (delta 959), reused 1230 (delta 799), pack-reused 55937
Receiving objects: 100% (57367/57367), 572.89 MiB | 14.11 MiB/s, done.
Resolving deltas: 100% (39624/39624), done.
Checking out files: 100% (2467/2467), done.


In [5]:
%cd /gdrive/MyDrive/colabfiles/tf2programmfolder/models/research

/gdrive/MyDrive/colabfiles/tf2programmfolder/models/research


In [6]:
!protoc object_detection/protos/*.proto --python_out=.

In [7]:
!cp object_detection/packages/tf2/setup.py .

In [ ]:
!python -m pip install .

In [9]:
!rm -r /gdrive/MyDrive/colabfiles/tf2programmfolder/models/research/object_detection/images

rm: cannot remove '/gdrive/MyDrive/colabfiles/tf2programmfolder/models/research/object_detection/images': No such file or directory


In [10]:
!mkdir /gdrive/MyDrive/colabfiles/tf2programmfolder/models/research/object_detection/images

In [11]:
!cp -r /gdrive/My\ Drive/colabfiles/dataset/20200916/train/ /gdrive/MyDrive/colabfiles/tf2programmfolder/models/research/object_detection/images

In [12]:
!cp -r /gdrive/My\ Drive/colabfiles/dataset/20200916/test/ /gdrive/MyDrive/colabfiles/tf2programmfolder/models/research/object_detection/images

In [13]:
!cp /gdrive/My\ Drive/colabfiles/dataset/20200916/test_labels.csv /gdrive/MyDrive/colabfiles/tf2programmfolder/models/research/object_detection/images
!cp /gdrive/My\ Drive/colabfiles/dataset/20200916/train_labels.csv /gdrive/MyDrive/colabfiles/tf2programmfolder/models/research/object_detection/images

In [14]:
%cd /gdrive/MyDrive/colabfiles/tf2programmfolder/models/research/object_detection
!git clone https://github.com/aalpatya/detect_hands.git

/gdrive/MyDrive/colabfiles/tf2programmfolder/models/research/object_detection
Cloning into 'detect_hands'...
remote: Enumerating objects: 206, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 206 (delta 38), reused 56 (delta 20), pack-reused 118
Receiving objects: 100% (206/206), 53.76 MiB | 13.48 MiB/s, done.
Resolving deltas: 100% (81/81), done.
Checking out files: 100% (100/100), done.


In [15]:
!mkdir TFLite_model_210525

In [16]:
%%writefile generate_tfrecord.py

"""
ORIGINAL SOURCE: https://github.com/datitran/raccoon_dataset/master/generate_tfrecord.py
All I have done here is to change line 19 from import tensorflow as tf 
to import tensorflow.compat.v1 as tf
and line 36 from racoon to hand
Usage:
  # From tensorflow/models/
  # Create train data:
  python generate_tfrecord.py --csv_input=data/train_labels.csv  --output_path=train.record

  # Create test data:
  python generate_tfrecord.py --csv_input=data/test_labels.csv  --output_path=test.record
"""
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import io
import pandas as pd
import tensorflow.compat.v1 as tf

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

flags = tf.app.flags
flags.DEFINE_string('csv_input', '', 'Path to the CSV input')
flags.DEFINE_string('output_path', '', 'Path to output TFRecord')
flags.DEFINE_string('image_dir', '', 'Path to images')
FLAGS = flags.FLAGS


# TO-DO replace this with label map
def class_text_to_int(row_label):
    if row_label == 'rasti':
        return 1
    else:
        None


def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


def main(_):
    writer = tf.python_io.TFRecordWriter(FLAGS.output_path)
    path = os.path.join(FLAGS.image_dir)
    examples = pd.read_csv(FLAGS.csv_input)
    grouped = split(examples, 'filename')
    for group in grouped:
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())

    writer.close()
    output_path = os.path.join(os.getcwd(), FLAGS.output_path)
    print('Successfully created the TFRecords: {}'.format(output_path))


if __name__ == '__main__':
    tf.app.run()

Writing generate_tfrecord.py


In [17]:
!python generate_tfrecord.py --csv_input=images/train_labels.csv --image_dir=images/train --output_path=train.record

2021-05-25 12:10:47.199805: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Successfully created the TFRecords: /gdrive/My Drive/colabfiles/tf2programmfolder/models/research/object_detection/train.record


In [18]:
!python generate_tfrecord.py --csv_input=images/test_labels.csv --image_dir=images/test --output_path=test.record

2021-05-25 12:10:53.945074: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Successfully created the TFRecords: /gdrive/My Drive/colabfiles/tf2programmfolder/models/research/object_detection/test.record


In [19]:
%cd /gdrive/MyDrive/colabfiles/tf2programmfolder/models/research/object_detection

/gdrive/MyDrive/colabfiles/tf2programmfolder/models/research/object_detection


In [20]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
# Unzip
!tar -xzvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

--2021-05-25 12:12:32--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.69.128, 2607:f8b0:4001:c08::80
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.69.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20515344 (20M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’

ssd_mobilenet_v2_fp 100%[===================>]  19.56M  47.5MB/s    in 0.4s    

2021-05-25 12:12:33 (47.5 MB/s) - ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’ saved [20515344/20515344]

ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/checkpoint
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/

In [21]:
%%writefile label_map.pbtxt

item {
  id: 1
  name: 'rasti'
}


Writing label_map.pbtxt


In [23]:
%%writefile pipeline.config

model {
  ssd {
    num_classes: 1
    image_resizer {
      fixed_shape_resizer {
        height: 320
        width: 320
      }
    }
    feature_extractor {
      type: "ssd_mobilenet_v2_fpn_keras"
      depth_multiplier: 1.0
      min_depth: 16
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 4e-05
          }
        }
        initializer {
          random_normal_initializer {
            mean: 0.0
            stddev: 0.01
          }
        }
        activation: RELU_6
        batch_norm {
          decay: 0.997
          scale: true
          epsilon: 0.001
        }
      }
      use_depthwise: true
      override_base_feature_extractor_hyperparams: true
      fpn {
        min_level: 3
        max_level: 7
        additional_layer_depth: 128
      }
    }
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
        use_matmul_gather: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    box_predictor {
      weight_shared_convolutional_box_predictor {
        conv_hyperparams {
          regularizer {
            l2_regularizer {
              weight: 4e-05
            }
          }
          initializer {
            random_normal_initializer {
              mean: 0.0
              stddev: 0.01
            }
          }
          activation: RELU_6
          batch_norm {
            decay: 0.997
            scale: true
            epsilon: 0.001
          }
        }
        depth: 128
        num_layers_before_predictor: 4
        kernel_size: 3
        class_prediction_bias_init: -4.6
        share_prediction_tower: true
        use_depthwise: true
      }
    }
    anchor_generator {
      multiscale_anchor_generator {
        min_level: 3
        max_level: 7
        anchor_scale: 4.0
        aspect_ratios: 1.0
        aspect_ratios: 2.0
        aspect_ratios: 0.5
        scales_per_octave: 2
      }
    }
    post_processing {
      batch_non_max_suppression {
        score_threshold: 1e-08
        iou_threshold: 0.6
        max_detections_per_class: 100
        max_total_detections: 100
        use_static_shapes: false
      }
      score_converter: SIGMOID
    }
    normalize_loss_by_num_matches: true
    loss {
      localization_loss {
        weighted_smooth_l1 {
        }
      }
      classification_loss {
        weighted_sigmoid_focal {
          gamma: 2.0
          alpha: 0.25
        }
      }
      classification_weight: 1.0
      localization_weight: 1.0
    }
    encode_background_as_zeros: true
    normalize_loc_loss_by_codesize: true
    inplace_batchnorm_update: true
    freeze_batchnorm: false
  }
}
train_config {
  batch_size: 4
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
  data_augmentation_options {
    random_crop_image {
      min_object_covered: 0.0
      min_aspect_ratio: 0.75
      max_aspect_ratio: 3.0
      min_area: 0.75
      max_area: 1.0
      overlap_thresh: 0.0
    }
  }
  sync_replicas: true
  optimizer {
    momentum_optimizer {
      learning_rate {
        cosine_decay_learning_rate {
          learning_rate_base: 0.08
          total_steps: 50000
          warmup_learning_rate: 0.026666
          warmup_steps: 1000
        }
      }
      momentum_optimizer_value: 0.9
    }
    use_moving_average: false
  }
  fine_tune_checkpoint: "/gdrive/MyDrive/colabfiles/tf2programmfolder/models/research/object_detection/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0"
  num_steps: 50000
  startup_delay_steps: 0.0
  replicas_to_aggregate: 8
  max_number_of_boxes: 100
  unpad_groundtruth_tensors: false
  fine_tune_checkpoint_type: "detection"
  fine_tune_checkpoint_version: V2
}
train_input_reader {
  label_map_path: "/gdrive/MyDrive/colabfiles/tf2programmfolder/models/research/object_detection/label_map.pbtxt"
  tf_record_input_reader {
    input_path: "/gdrive/MyDrive/colabfiles/tf2programmfolder/models/research/object_detection/train.record"
  }
}
eval_config {
  metrics_set: "coco_detection_metrics"
  use_moving_averages: false
}
eval_input_reader {
  label_map_path: "/gdrive/MyDrive/colabfiles/tf2programmfolder/models/research/object_detection/label_map.pbtxt"
  shuffle: false
  num_epochs: 1
  tf_record_input_reader {
    input_path: "/gdrive/MyDrive/colabfiles/tf2programmfolder/models/research/object_detection/test.record"
  }
}

Overwriting pipeline.config


In [ ]:
%load_ext tensorboard
%tensorboard --logdir=/content/output_training/train

In [ ]:
%cd /content/models/research/object_detection/

/content/models/research/object_detection


In [ ]:
%ls

In [25]:
#train 
!python model_main_tf2.py \
--pipeline_config_path=pipeline.config \
--model_dir=output_training --alsologtostderr

^C


In [26]:
!python exporter_main_v2.py \
--trained_checkpoint_dir=output_training \
--pipeline_config_path=pipeline.config \
--output_directory TFLite_model_210525

2021-05-25 12:32:16.021929: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-05-25 12:32:18.325188: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-05-25 12:32:18.348122: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-05-25 12:32:18.348932: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2021-05-25 12:32:18.349010: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-05-25 12:32:18.352132: I tensorflow/stream_executor/platform/defa

In [27]:
!mkdir /gdrive/MyDrive/colabfiles/tf2programmfolder/models/research/object_detection/TFLite_model_210525/endresult

In [29]:
!python export_tflite_graph_tf2.py \
    --pipeline_config_path /gdrive/MyDrive/colabfiles/tf2programmfolder/models/research/object_detection/TFLite_model_210525/pipeline.config \
    --trained_checkpoint_dir /gdrive/MyDrive/colabfiles/tf2programmfolder/models/research/object_detection/TFLite_model_210525/checkpoint \
    --output_directory /gdrive/MyDrive/colabfiles/tf2programmfolder/models/research/object_detection/TFLite_model_210525/endresult

2021-05-25 12:37:22.903153: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-05-25 12:37:25.302791: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-05-25 12:37:25.324388: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-05-25 12:37:25.325197: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2021-05-25 12:37:25.325248: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-05-25 12:37:25.328461: I tensorflow/stream_executor/platform/defa

In [30]:
!tflite_convert \
  --saved_model_dir=/gdrive/MyDrive/colabfiles/tf2programmfolder/models/research/object_detection/TFLite_model_210525/endresult/saved_model \
  --output_file=/gdrive/MyDrive/colabfiles/tf2programmfolder/models/research/object_detection/TFLite_model_210525/endresult/detect.tflite

2021-05-25 12:39:33.908046: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-05-25 12:39:36.015287: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-05-25 12:39:36.035993: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-05-25 12:39:36.036761: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2021-05-25 12:39:36.036810: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-05-25 12:39:36.041950: I tensorflow/stream_executor/platform/defa

# New Section